In [22]:
from math import log
import operator

In [2]:
#计算给定数据集的香农熵
def calcShannonEnt(dataSet):
    #计算每个类别出现次数
    numEntries = len(dataSet)
    labelCounts = {}
    for featVec in dataSet:
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    #根据类别次数计算熵
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key]) / numEntries
        shannonEnt -= prob * log(prob,2)
    return shannonEnt

In [3]:
#计算给定数据集的香农熵
def calcShannonEnt(dataSet):
    #计算每个类别出现次数
    numEntries = len(dataSet)
    labelCounts = {}
    for featVec in dataSet:
        currentLabel = featVec[-1]
#         if currentLabel not in labelCounts.keys():
        if currentLabel not in labelCounts:
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    #根据类别次数计算熵
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key]) / numEntries
        shannonEnt -= prob * log(prob,2)
    return shannonEnt

In [17]:
x = {}
help(x.get)

Help on built-in function get:

get(...) method of builtins.dict instance
    D.get(k[,d]) -> D[k] if k in D, else d.  d defaults to None.



In [4]:
#计算给定数据集的香农熵
def calcShannonEnt(dataSet):
    #计算每个类别出现次数
    numEntries = len(dataSet)
    labelCounts = {}
    for featVec in dataSet:
        currentLabel = featVec[-1]
        labelCounts[currentLabel] = labelCounts.get(currentLabel, 0) + 1
#         if currentLabel not in labelCounts:
#             labelCounts[currentLabel] = 0
#         labelCounts[currentLabel] += 1
    #根据类别次数计算熵
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key]) / numEntries
        shannonEnt -= prob * log(prob,2)
    return shannonEnt

In [36]:
#构造简单数据集
def createDataSet():
    dataSet = [
        [1, 1, 'yes'],
        [1, 1, 'yes'],
        [1, 0, 'no'],
        [0, 1, 'no'],
        [0, 1, 'no']
    ]
    labels = ['no surfacing', 'flippers']
    return dataSet,labels

In [37]:
myDat, labels = createDataSet()
myDat

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [38]:
calcShannonEnt(myDat)

0.9709505944546686

In [39]:
calcShannonEnt(myDat)

0.9709505944546686

In [9]:
#按照给定特征(axis)的值(value)划分数据集
def splitDataSet(dataSet,axis,value):
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:
            #删除已经判别的axis特征
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

In [40]:
splitDataSet(myDat,0,1)

[[1, 'yes'], [1, 'yes'], [0, 'no']]

In [20]:
#选择使得信息增益最大的划分特征
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0
    bestFeature = -1
    for i in range(numFeatures):
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet,i,value)
            prob = float(len(subDataSet))/len(dataSet)
            newEntropy -= prob * log(prob,2)
        infoGain = baseEntropy - newEntropy
        if(infoGain > bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature

In [41]:
chooseBestFeatureToSplit(myDat)

1

In [23]:
#叶子节点类标签不唯一（多数表决）
def majorityCnt(classList):
    classCount={}
    for vote in classList:
        classCount[vote] = classCount.get(classCount[vote],0) + 1
    sortedClassCount = sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)
    return sortedClassCount[0][0]

In [25]:
help(sorted)

Help on built-in function sorted in module builtins:

sorted(iterable, key=None, reverse=False)
    Return a new list containing all items from the iterable in ascending order.
    
    A custom key function can be supplied to customise the sort order, and the
    reverse flag can be set to request the result in descending order.



In [33]:
#创建树的函数代码
def createTree(dataSet,labels):
    classList = [example[-1] for example in dataSet]
    #划分数据集两个终止条件：当前数据集标签纯（返回此标签）；当前数据集特征使用完标签仍不纯（多数表决）
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    if len(dataSet) == 1:
        return majorityCnt(classList)
    #最好特征的索引值
    bestFeat = chooseBestFeatureToSplit(dataSet)
    #最好特征的值
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet,bestFeat,value),subLabels)
    return myTree

In [42]:
myTree = createTree(myDat,labels)

In [43]:
myTree

{'flippers': {0: 'no', 1: {'no surfacing': {'no': 'no', 'yes': 'yes'}}}}